<a href="https://colab.research.google.com/github/Preetibanga18/githubactions-codespace/blob/main/AI_Fitness_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
from cv2 import destroyAllWindows
import mediapipe as mp
import numpy as np
import warnings
warnings.filterwarnings('ignore')
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

ModuleNotFoundError: No module named 'mediapipe'

# Squat

In [ ]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle >180.0:
        angle = 360-angle

    return angle

In [ ]:
def rescale_frame(frame, percent=50):
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

In [ ]:
angle_min = []
angle_min_hip = []

cap = cv2.VideoCapture("/Users/shriyansh/Downloads/squat.mp4")
# Curl counter variables
counter = 0
min_ang = 0
max_ang = 0
min_ang_hip = 0
max_ang_hip = 0
stage = None

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) + 0.5)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) + 0.5)
size = (640, 480)
fourcc = cv2.VideoWriter_fourcc(*'MP4V')

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if frame is not None:
            frame_ = rescale_frame(frame, percent=55)

        # Recolor image to RGB
        image = cv2.cvtColor(frame_, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            # Calculate angle
            angle_knee = calculate_angle(hip, knee, ankle)  # Knee joint angle
            angle_knee = round(angle_knee, 2)

            angle_hip = calculate_angle(shoulder, hip, knee)  # Hip joint angle
            angle_hip = round(angle_hip, 2)

            hip_angle = 180 - angle_hip
            knee_angle = 180 - angle_knee

            angle_min.append(angle_knee)
            angle_min_hip.append(angle_hip)

            print(angle_knee)

            # Suggestions based on the angle calculation
            suggestion = ""
            if angle_knee > 150:
                suggestion = "Lower your knees more for a proper squat."
            elif angle_knee < 90:
                suggestion = "Do not bend your knees too much, try to control the descent."

            if angle_hip < 90:
                suggestion = "Try to keep your hips more aligned during the squat."

            # Display suggestion on screen
            cv2.putText(image, suggestion,
                        (30,100),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)

            # Visualize angles
            cv2.putText(image, str(angle_knee),
                           tuple(np.multiply(knee, [630, 900]).astype(int)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 9, 0), 2, cv2.LINE_AA)

            cv2.putText(image, str(angle_hip),
                           tuple(np.multiply(hip, [630, 900]).astype(int)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)

            # Curl counter logic
            if angle_knee > 150:
                stage = "up"
            if angle_knee <= 100 and stage == 'up':
                stage = "down"
                counter += 1
                print(counter)
                min_ang = min(angle_min)
                max_ang = max(angle_min)
                min_ang_hip = min(angle_min_hip)
                max_ang_hip = max(angle_min_hip)

                print(min(angle_min), " _ ", max(angle_min))
                print(min(angle_min_hip), " _ ", max(angle_min_hip))

                # Reset lists for the next rep
                angle_min = []
                angle_min_hip = []
        except:
            pass

        # Render squat counter
        # Setup status box
        cv2.rectangle(image, (20, 20), (200, 90), (0, 0, 0), -1)

        # Rep data
        cv2.putText(image, "Repetition : " + str(counter),
                    (30, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1, cv2.LINE_AA)

        # Knee angle:
        cv2.putText(image, "Knee-joint angle : " + str(min_ang),
                    (30, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1, cv2.LINE_AA)

        # Hip angle:
        cv2.putText(image, "Hip-joint angle : " + str(min_ang_hip),
                    (30, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1, cv2.LINE_AA)

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=1),
                                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=1))

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()

    cap.release()
    cv2.destroyAllWindows()


I0000 00:00:1743579951.326899  477330 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M2
W0000 00:00:1743579951.404638  615510 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743579951.429784  615513 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


179.28
176.56
179.48
169.74
141.88
114.44
120.04
108.84
116.45
108.07
104.65
105.21
110.43
115.19
120.89
125.4
120.84
126.98
128.74
135.26
137.32
148.57
144.25
134.14
133.71
135.28
146.12
149.57
150.25
139.69
138.37
145.88
148.86
149.65
155.19
154.33
158.82
165.34
163.7
162.59
158.06
151.8
156.66
152.53
149.74
147.57
152.88
146.92
152.21
137.62
144.64
154.38
151.6
148.58
152.89
151.3
144.01
148.26
147.84
152.02
154.09
144.26
146.99
153.72
159.27
159.0
163.64
140.54
148.8
140.45
135.16
146.56
153.28
169.22
152.5
158.38
151.74
156.44
178.71
173.52
158.52
147.81
162.72
165.42
153.81
145.97
160.13
158.01
130.43
166.11
161.68
140.55
129.91
122.85
139.29
126.94
133.17
137.87
138.99
142.32
144.73
147.08
152.26
159.57
158.97
163.51
164.96
161.19
158.66
141.08
132.85
135.41
131.93
132.1
124.34
148.29
125.86
116.58
133.57
132.14
84.34
1
84.34  _  179.48
116.33  _  179.94
72.91
86.23
56.12
53.96
41.18
42.1
41.1
34.59
37.61
34.76
35.1
35.07
31.53
30.61
31.44
35.14
37.45
32.84
30.92
29.72
29.42
28.

# Bicep Curl

In [ ]:
# Initialize variables
flag_left = None     # Flag for left hand position (Either UP or DOWN)
flag_right = None    # Flag for right hand position (Either UP or DOWN)
count_left = 0       # Count for left hand curls
count_right = 0      # Count for right hand curls

cap = cv2.VideoCapture("/Users/shriyansh/Downloads/5319439-uhd_2160_3840_25fps.mp4")
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.5)  # Landmark detection model instance

def calc_angle(a, b, c):
    """Calculate the angle between three points a, b, c."""
    a = np.array([a.x, a.y])  # Point A
    b = np.array([b.x, b.y])  # Point B (vertex)
    c = np.array([c.x, c.y])  # Point C

    # Calculate the angle
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    # Ensure the angle is between 0 and 180 degrees
    if angle > 180:
        angle = 360 - angle

    return angle

while cap.isOpened():
    _, frame = cap.read()

    # BGR to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR frame to RGB
    image.flags.writeable = False

    # Make Detections
    results = pose.process(image)  # Get landmarks of the object in frame from the model

    # Back to BGR
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Convert RGB back to BGR

    try:
        # Extract Landmarks
        landmarks = results.pose_landmarks.landmark

        # Left arm landmarks
        shoulder_left = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
        elbow_left = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW]
        wrist_left = landmarks[mp_pose.PoseLandmark.LEFT_WRIST]

        # Right arm landmarks
        shoulder_right = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]
        elbow_right = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW]
        wrist_right = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST]

        # Calculate angles for both elbows
        angle_left = calc_angle(shoulder_left, elbow_left, wrist_left)  # Left elbow angle
        angle_right = calc_angle(shoulder_right, elbow_right, wrist_right)  # Right elbow angle

        # Visualize angle for left elbow
        cv2.putText(image, f'Left Angle: {int(angle_left)}',
                    tuple(np.multiply([elbow_left.x, elbow_left.y], [640, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Visualize angle for right elbow
        cv2.putText(image, f'Right Angle: {int(angle_right)}',
                    tuple(np.multiply([elbow_right.x, elbow_right.y], [640, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Counter Logic for Left Arm
        if angle_left > 160:
            flag_left = 'down'
        if angle_left < 80 and flag_left == 'down':
            count_left += 1
            flag_left = 'up'

        # Counter Logic for Right Arm
        if angle_right > 160:
            flag_right = 'down'
        if angle_right < 80 and flag_right == 'down':
            count_right += 1
            flag_right = 'up'

    except:
        pass

    # Setup Status Box for Reps
    cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
    cv2.putText(image, f'Left Curls: {count_left}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(image, f'Right Curls: {count_right}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

    # Render Detections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Show the frame with angle and rep count for both arms
    cv2.imshow('MediaPipe Feed', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1743584334.216656  477330 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M2
W0000 00:00:1743584334.298586  664750 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743584334.310347  664750 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


# Deadlift

In [ ]:
# Initialize variables for counting reps
counter = 0
stage = None
angle_min = []

# Open the video capture
cap = cv2.VideoCapture("/Users/shriyansh/Downloads/12890962-uhd_2160_3840_60fps.mp4")

def calculate_angle(a, b, c):
    """Calculate the angle between three points a, b, c."""
    a = np.array(a)  # Point A
    b = np.array(b)  # Point B (vertex)
    c = np.array(c)  # Point C

    # Calculate the angle
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    # Ensure the angle is between 0 and 180 degrees
    if angle > 180:
        angle = 360 - angle

    return angle

# Start the pose detection model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to RGB and process it
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates of hip, knee, and shoulder
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]

            # Calculate the angle at the hip-knee-shoulder
            angle = calculate_angle(hip, knee, shoulder)

            # Track the "down" and "up" stages based on the angle
            if angle > 50:  # Threshold angle for "down" position
                stage = "down"
            if angle <30 and stage == 'down':  # Threshold angle for "up" position
                stage = "up"
                counter += 1
                print(f"Reps: {counter}")

            # Visualize the angle on the frame
            cv2.putText(image, f'Angle: {int(angle)}',
                        tuple(np.multiply(knee, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # Render the pose landmarks
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        except:
            pass

        # Display the repetition count and angle information
        cv2.rectangle(image, (20, 20), (200, 90), (0, 0, 0), -1)
        cv2.putText(image, f'Reps: {counter}', (30, 140), cv2.FONT_HERSHEY_SIMPLEX, 3.4, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(image, f'Angle: {int(angle)}', (30, 190), cv2.FONT_HERSHEY_SIMPLEX, 3.4, (255, 255, 255), 1, cv2.LINE_AA)

        # Show the frame
        cv2.imshow('Deadlift Counter', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


I0000 00:00:1743583084.159297  477330 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M2
W0000 00:00:1743583084.267186  649599 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743583084.279702  649600 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


# Pull ups

In [ ]:

# Initialize variables
flag_left = None     # Flag for left hand position (Either UP or DOWN)
count = 0       # Count for left pull-ups

cap = cv2.VideoCapture("/Users/shriyansh/Downloads/david goggins pull up videos.mp4")
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.5)  # Landmark detection model instance

def calc_angle(a, b, c):
    """Calculate the angle between three points a, b, c."""
    a = np.array([a.x, a.y])  # Point A
    b = np.array([b.x, b.y])  # Point B (vertex)
    c = np.array([c.x, c.y])  # Point C

    # Calculate the angle
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    # Ensure the angle is between 0 and 180 degrees
    if angle > 180:
        angle = 360 - angle

    return angle

while cap.isOpened():
    _, frame = cap.read()

    # BGR to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR frame to RGB
    image.flags.writeable = False

    # Make Detections
    results = pose.process(image)  # Get landmarks of the object in frame from the model

    # Back to BGR
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Convert RGB back to BGR

    try:
        # Extract Landmarks
        landmarks = results.pose_landmarks.landmark

        #  arm landmarks
        shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
        elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW]
        wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST]

        # Calculate angle for the  elbow
        angle = calc_angle(shoulder, elbow, wrist)  #  elbow angle

        # Visualize angle for  elbow
        cv2.putText(image, f' Angle: {int(angle)}',
                    tuple(np.multiply([elbow.x, elbow.y], [640, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Counter Logic
        if angle > 140:  # Arms fully extended (up position)
            flag = 'down'
        if angle < 65 and flag == 'down':  # Elbows bent (down position)
            count += 1
            flag = 'up'

    except:
        pass

    # Setup Status Box for Reps
    cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
    cv2.putText(image, f' Pull-ups: {count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

    # Render Detections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Show the frame with angle and rep count for the left arm
    cv2.imshow('MediaPipe Feed', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1743586132.724852  477330 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M2
W0000 00:00:1743586132.922732  685853 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743586132.941623  685853 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
